<a href="https://colab.research.google.com/github/26medias/TF-Face-Angle-Translation/blob/master/Face_Position_Dataset_Builder_One_shot_per_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Face Angle Dataset Generator


## Credits

Face extraction built thanks to https://machinelearningmastery.com/how-to-perform-face-recognition-with-vggface2-convolutional-neural-network-in-keras/

## How this works

1. Download movie trailers
2. Extract the frames from the video files
3. Extract the faces from the images
4. Cluster the faces by actor
5. Build & save the facial landmarks for each face
6. Build the dataset
7. Zip & upload the dataset to Google Storage

## Downloading videos, extracting the frames

We're going to download movie trailers from https://www.davestrailerpage.co.uk/

The frames from the video files will be extracted and saved to file.

## Code setup: Imports & methods

Pip install

In [1]:
!pip install git+https://github.com/rcmalli/keras-vggface.git
!pip show keras-vggface
!pip install matplotlib
!pip install mtcnn

  Cloning https://github.com/rcmalli/keras-vggface.git to /tmp/pip-req-build-8xmundr0
  Running command git clone -q https://github.com/rcmalli/keras-vggface.git /tmp/pip-req-build-8xmundr0
  Created wheel for keras-vggface: filename=keras_vggface-0.6-cp36-none-any.whl size=8311 sha256=5596731204f2ce9269abff348bd52d56d99a768cd98f26750e00741fec025ca0
  Stored in directory: /tmp/pip-ephem-wheel-cache-rw8xlel4/wheels/36/07/46/06c25ce8e9cd396dabe151ea1d8a2bc28dafcb11321c1f3a6d
Successfully built keras-vggface
Name: keras-vggface
Version: 0.6
Summary: VGGFace implementation with Keras framework
Home-page: https://github.com/rcmalli/keras-vggface
Author: Refik Can MALLI
Author-email: mallir@itu.edu.tr
License: MIT
Location: /usr/local/lib/python3.6/dist-packages
Requires: numpy, pillow, six, pyyaml, h5py, scipy, keras
Required-by: 
     |████████████████████████████████| 2.3MB 31.2MB/s 
  Created wheel for mtcnn: filename=mtcnn-0.0.9-cp36-none-any.whl size=2257690 sha256=c945ccbedff031413f1f

Code

In [0]:
!rm -r faces/0

In [0]:
import requests
import ntpath
import cv2
import math
import os, sys
from matplotlib import pyplot
from PIL import Image
import numpy as np
from numpy import asarray
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
import keras_vggface
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input
import glob
import mtcnn
from pathlib import Path
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from scipy.cluster import  hierarchy

# create the detector, using default weights
detector = MTCNN()

# create a vggface model
embedding_model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')

# The variables
DIR_VIDEOS = "videos"
DIR_IMAGES = "images"
DIR_FACES = "faces"
CAPTURE_FPS  = 23*20 # We'll extract 30 images per second of video

if not os.path.isdir(DIR_VIDEOS):
  os.mkdir(DIR_VIDEOS, 755);
if not os.path.isdir(DIR_IMAGES):
  os.mkdir(DIR_IMAGES, 755);
if not os.path.isdir(DIR_FACES):
  os.mkdir(DIR_FACES, 755);

# The methods
# ===========

# Get the directory of a filename
def getDir(filename):
  p = Path(filename);
  return p.parts[len(p.parts)-2]
# Dowload a video from a url
def downloadFile(url):
  myfile = requests.get(url)
  filename = DIR_VIDEOS+"/"+ntpath.basename(url)
  open(filename, 'wb').write(myfile.content)
  return filename


# Extract the faces from an image, return an array of numpy faces
def extractFacesFromImage(pixels, required_size=(224, 224), limit=50):
  results = detector.detect_faces(pixels)
  faces = []
  for i,faceData in enumerate(results):
    if len(faces) > limit:
      break
    x1, y1, width, height = faceData['box']
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    try:
      image = Image.fromarray(face)
      image = image.resize(required_size)
      face_array = asarray(image)
      faces.append(face_array)
      if limit==1:
        return face_array
    except:
      print("Face processing failed")
  if limit==1 and len(faces)==0:
    return False
  return faces;


# Export the frames out of a video at a specific fps
def videoToFaces(filename, skipFrame=10):
  basename = os.path.splitext(ntpath.basename(filename))[0]
  print("basename:", basename)
  if not os.path.isdir(DIR_IMAGES+"/"+basename):
    os.mkdir(DIR_IMAGES+"/"+basename, 755)
  cap = cv2.VideoCapture(filename)
  # Get the video's FPS
  fps = cap.get(cv2.CAP_PROP_FPS)
  print(basename, ": fps: ",fps," / skipFrame: ", skipFrame)
  i = 0
  c = 0
  faces = []
  while(cap.isOpened()):
      ret, frame = cap.read()
      if ret == False:
          break
      i+=1
      if (i % skipFrame == 0):
        continue
      frameFaces = extractFacesFromImage(frame)
      for f in frameFaces:
        faces.append(f)
        c+=1
      #cv2.imwrite(DIR_IMAGES+"/"+basename+'/'+str(round((i-1)/fps,2))+'sec.jpg',frame)
  cap.release()
  cv2.destroyAllWindows()
  print(basename, " processed.")
  print(c,"/",i," frames analyzed.")
  print(len(faces), " faces found.")
  return faces


# Show a few images
def showImages(images, width=4):
  fig = pyplot.figure(figsize=(width, math.ceil(len(images)/width)))
  for i in range(len(images)):
      pyplot.subplot(width, math.ceil(len(images)/width), i+1)
      pyplot.imshow(images[i])
      pyplot.axis('off')
  pyplot.savefig('preview.png')
  pyplot.show()

#Save an array of images to files
def saveImages(images, dest):
  for n, image in enumerate(images):
    cv2.imwrite(dest+"/"+str(n)+'.jpg', image)

# Extract faces and calculate face embeddings for a list of photo files
def get_embeddings(faces):
	# convert into an array of samples
	samples = asarray(faces, 'float32')
	# prepare the face for the model, e.g. center pixels
	samples = preprocess_input(samples, version=2)
	# perform prediction
	embeddings = embedding_model.predict(samples)
	return embeddings


# Determine if a candidate face is a match for a known face
def is_match(known_embedding, candidate_embedding, threshold=0.5):
	# calculate distance between embeddings
	score = cosine(known_embedding, candidate_embedding)
	return score >= threshold

# Cluster the faces by cosine distance
def clusterFaces(faces, embeddings):
  print("Clustering the faces...")
  groups = []
  for n, face in enumerate(faces):
    if len(groups)==0:
      print("First face in the cluster")
      groups.append({
        "faces":     [face],
        "embeddings":[embeddings[n]]
      })
    else:
      scores = []
      for g, group in enumerate(groups):
        for embedding in group["embeddings"]:
          scores.append(cosine(embedding, embeddings[n]))
        score = np.mean(scores)
        #print("face #", n, " group #", g, "score:", score)
        scores.append({
            "group": g,
            "score": score
        })
      scores = sorted(scores, key = lambda i: i["score"], reverse=True) 
      print("face #", n, " group #", scores[0].group, "score:", scores[0].score)
      if scores[0]["score"] >= 0.6:
        groups[scores[0].group]["embeddings"].append(embeddings[n])
        groups[scores[0].group]["faces"].append(face)
      else:
        groups.append({
          "faces":     [face],
          "embeddings":[embeddings[n]]
        })
  return groups;

# Cluster all the faces from a remote video
def clusterFacesOnVideo(url):
  videoFilename = downloadFile(url)
  faces         = videoToFaces(videoFilename, CAPTURE_FPS)
  
  if not os.path.isdir(DIR_FACES+"/ALL"):
    os.mkdir(DIR_FACES+"/ALL", 755);
  #saveImages(faces, DIR_FACES+"/ALL")
  
  embeddings    = get_embeddings(faces)
  clusters      = clusterFaces(faces, embeddings)
  for n, group in enumerate(clusters):
    if not os.path.isdir(DIR_FACES+"/"+str(n)):
      os.mkdir(DIR_FACES+"/"+str(n), 755);
    saveImages(group["faces"], DIR_FACES+"/"+str(n))
  #showImages(faces)

#remoteVideoToImages("http://trailers.apple.com/movies/paramount/terminator-dark-fate/terminator-dark-fate-trailer-2_h480p.mov")
#extractFacesFromDirectory(DIR_IMAGES, DIR_FACES)

In [43]:
scores = []
for i in range(5):
  scores.append({
      "i": i,
      "v": i*5
  })
scores = sorted(scores, key = lambda i: i['i'], reverse=False) 
print(scores)

[{'i': 0, 'v': 0}, {'i': 1, 'v': 5}, {'i': 2, 'v': 10}, {'i': 3, 'v': 15}, {'i': 4, 'v': 20}]


## Execution

In [47]:
clusterFacesOnVideo("http://trailers.apple.com/movies/paramount/terminator-dark-fate/terminator-dark-fate-trailer-2_h480p.mov")

basename: terminator-dark-fate-trailer-2_h480p
terminator-dark-fate-trailer-2_h480p : fps:  23.976023976023978  / skipFrame:  460
Face processing failed
Face processing failed
Face processing failed
Face processing failed
Face processing failed
Face processing failed
Face processing failed
Face processing failed
Face processing failed
Face processing failed
Face processing failed
Face processing failed
Face processing failed
Face processing failed
Face processing failed
Face processing failed
Face processing failed
Face processing failed
Face processing failed
Face processing failed
Face processing failed
terminator-dark-fate-trailer-2_h480p  processed.
1044 / 3774  frames analyzed.
1044  faces found.
Clustering the faces...
First face in the cluster


IndexError: ignored

In [0]:
!tar -zcvf faces.tar.gz faces

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
!gcloud config set project deep-learning-files
#!gsutil cp gs://tf-face-angle-translation/foo.bar ./foo.bar
!gsutil cp  ./faces.tar.gz gs://tf-face-angle-translation/datasets/faces-terminator.tar.gz